# Explainability Extract Analysis

This notebook demonstrates the `DecisionAnalyzer` class from pdstools using the Explainability Extract v1 dataset.

The Explainability Extract captures actions at the **Arbitration** stage only. It can be extracted from Infinity 24.1 and earlier versions. For the full decision funnel (v2 format), see the [Decision Analyzer notebook](../decision_analyzer/decision_analyzer.ipynb).

The analyses below cover action distribution, sensitivity, win/loss patterns, and personalization.

In [ ]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import plotly.io as pio

pio.renderers.default = "notebook_connected"

In [ ]:
from pdstools.decision_analyzer.data_read_utils import read_data
from pdstools.decision_analyzer.DecisionAnalyzer import DecisionAnalyzer
from pdstools import read_ds_export
import polars as pl

### Read Data

Load the sample Explainability Extract data and create a `DecisionAnalyzer` instance. The sample data can be downloaded directly from [GitHub](https://github.com/pegasystems/pega-datascientist-tools/raw/master/data/sample_explainability_extract.parquet).

In [ ]:
df = read_ds_export(
    filename="sample_explainability_extract.parquet",
    path="https://raw.githubusercontent.com/pegasystems/pega-datascientist-tools/master/data",
)
decision_data = DecisionAnalyzer(df)

In [ ]:
decision_data.decision_data.collect()

### Overview

General statistics of the dataset.

In [ ]:
decision_data.get_overview_stats

A single decision. The number of rows shows how many actions are available at the Arbitration stage. `Rank` shows the action ranking.

In [ ]:
selected_interaction_id = (
    decision_data.unfiltered_raw_decision_data.select("Interaction ID")
    .first()
    .collect()
    .row(0)[0]
)
print(f"{selected_interaction_id=}")
decision_data.unfiltered_raw_decision_data.filter(
    pl.col("Interaction ID") == selected_interaction_id
).sort("Rank").collect()

### Action Distribution

Distribution of actions at the Arbitration stage. Helps identify action groups that rarely survive to Arbitration.

In [ ]:
stage = "Arbitration"
scope_options = ["Issue", "Group", "Action"]
distribution_data = decision_data.getDistributionData(stage, scope_options)
fig = decision_data.plot.distribution_as_treemap(
    df=distribution_data, stage=stage, scope_options=scope_options
)
fig.show()

### Global Sensitivity

Shows the impact of each arbitration factor (Propensity, Value, Context Weight, Levers) on the final decision. Each bar represents how many decisions would change if that factor were removed. Ideally, Propensity should have the strongest influence.

In [ ]:
decision_data.plot.sensitivity(win_rank=1)

### Wins and Losses in Arbitration

Distribution of wins and losses by Issue. The `level` parameter can be set to `"Group"` or `"Action"` for different granularity. Actions are classified as winning or losing based on `win_rank`.

In [ ]:
decision_data.plot.global_winloss_distribution(level="Issue", win_rank=1)

### Personalization Analysis

Shows how many actions are available per customer at the Arbitration stage. Limited optionality reduces the ability to personalize. The bars show decision counts per number of available actions; the line shows average propensity of the top-ranked action. Average propensity should increase with more available actions.

In [ ]:
decision_data.plot.propensity_vs_optionality(stage="Arbitration")

### Win/Loss Analysis

#### Win Analysis
Select an action and see how often it wins and which actions it defeats.

In [ ]:
win_rank = 1
selected_action = (
    decision_data.unfiltered_raw_decision_data.filter(pl.col("Rank") == 1)
    .group_by("Action")
    .len()
    .sort("len", descending=True)
    .collect()
    .get_column("Action")
    .to_list()[1]
)
filter_statement = pl.col("Action") == selected_action

interactions_where_comparison_group_wins = (
    decision_data.get_winning_or_losing_interactions(
        win_rank=win_rank,
        group_filter=filter_statement,
        win=True,
    )
)

print(
    f"selected action '{selected_action}' wins(Rank{win_rank}) in {interactions_where_comparison_group_wins.collect().height} interactions."
)

Actions that lose to the selected action in arbitration.

In [ ]:
# Losing actions in interactions where the selected action wins.
groupby_cols = ["Issue", "Group", "Action"]
winning_from = decision_data.winning_from(
    interactions=interactions_where_comparison_group_wins,
    win_rank=win_rank,
    groupby_cols=groupby_cols,
    top_k=20,
)

decision_data.plot.distribution_as_treemap(
    df=winning_from, stage="Arbitration", scope_options=groupby_cols
)

#### Loss Analysis
Actions that beat the selected action in arbitration.

In [ ]:
interactions_where_comparison_group_loses = (
    decision_data.get_winning_or_losing_interactions(
        win_rank=win_rank,
        group_filter=filter_statement,
        win=False,
    )
)

print(
    f"selected action '{selected_action}' loses in {interactions_where_comparison_group_loses.collect().height} interactions."
)
# Winning actions in interactions where the selected action loses.
losing_to = decision_data.losing_to(
    interactions=interactions_where_comparison_group_loses,
    win_rank=win_rank,
    groupby_cols=groupby_cols,
    top_k=20,
)

decision_data.plot.distribution_as_treemap(
    df=losing_to, stage="Arbitration", scope_options=groupby_cols
)

#### Sensitivity for Selected Action
Change in win count when each prioritization factor is individually removed. Unlike the global sensitivity above, negative values are possible: a negative value means removing that factor would increase wins for the selected action (i.e., that factor is hurting it).

In [ ]:
decision_data.plot.sensitivity(
    reference_group=pl.col("Action") == selected_action
)

#### Prioritization Factor Distributions

Box plots comparing the arbitration factor distributions of the selected action vs competitors in the same interactions.

In [ ]:
fig, warning_message = decision_data.plot.prio_factor_boxplots(
    reference=pl.col("Action") == selected_action,
)
if warning_message:
    print(warning_message)
else:
    fig.show()

#### Rank Distribution

Distribution of the prioritization rank for the selected action. Low ranks indicate the action is not often winning.

In [ ]:
decision_data.plot.rank_boxplot(
    reference=pl.col("Action") == selected_action,
)